In [1]:
!pip list|grep albumentations

albumentations                1.3.1


In [4]:
import numpy as np
import torch
import torchaudio
import pandas as pd

def spectrogram_from_eeg4(parquet_path) -> np.ndarray:
    NAMES = ["LL", "LP", "RP", "RR"]

    FEATS = [
        ["Fp1", "F7", "T3", "T5", "O1"],
        ["Fp1", "F3", "C3", "P3", "O1"],
        ["Fp2", "F8", "T4", "T6", "O2"],
        ["Fp2", "F4", "C4", "P4", "O2"],
    ]
    
    FOUR = 4
    
    # Load middle 50 seconds of EEG series
    eeg = pd.read_parquet(parquet_path)
    middle = (len(eeg) - 10_000) // 2
    eeg = eeg.iloc[middle : middle + 10_000]

    # Variable to hold spectrogram
    img = np.zeros((128, 256, 4), dtype="float32")

    for k in range(4):
        cols = FEATS[k]

        for kk in range(4):
            # Compute pair differences
            x = eeg[cols[kk]].to_numpy() - eeg[cols[kk + 1]].to_numpy()

            # Fill NaNs
            m = np.nanmean(x)
            x = np.where(np.isnan(x), m, x)  # Vectorized operation for replacing NaNs

            # Convert to tensor and add a batch dimension
            x_tensor = torch.tensor(x, dtype=torch.float32).unsqueeze(0)

            # Create MelSpectrogram object
            mel_spectrogram = torchaudio.transforms.MelSpectrogram(
                sample_rate=200,
                n_fft=1024,
                win_length=128,
                hop_length=len(x) // 256,
                n_mels=128,
                f_min=0,
                f_max=20,
                power=2.0,
            )

            # Compute spectrogram
            mel_spec_tensor = mel_spectrogram(x_tensor)

            # Convert power spectrogram to dB scale
            mel_spec_db_tensor = torchaudio.transforms.AmplitudeToDB(stype="power")(mel_spec_tensor)

            # Ensure the spectrogram is the expected shape
            width = min(mel_spec_db_tensor.shape[2], 256)
            mel_spec_db_tensor = mel_spec_db_tensor[:, :, :width].squeeze(0)  # Remove batch dimension

            # Standardize to -1 to 1
            mel_spec_db_np = (mel_spec_db_tensor.numpy() + 40) / 40
            img[:, :width, k] += mel_spec_db_np

        # Average the 4 montage differences
        img[:, :width, k] /= 4.0

    return img[::-1]

In [5]:
eeg_file_path = "../data/1000086677.parquet"

img = spectrogram_from_eeg4(eeg_file_path)
print("Spectrogram shape = ", img.shape)
print(img.min(), img.max())

KeyError: 'Fp1'

In [7]:
pd.read_parquet(eeg_file_path).columns

Index(['time', 'LL_0.59', 'LL_0.78', 'LL_0.98', 'LL_1.17', 'LL_1.37',
       'LL_1.56', 'LL_1.76', 'LL_1.95', 'LL_2.15',
       ...
       'RP_18.16', 'RP_18.36', 'RP_18.55', 'RP_18.75', 'RP_18.95', 'RP_19.14',
       'RP_19.34', 'RP_19.53', 'RP_19.73', 'RP_19.92'],
      dtype='object', length=401)